In [31]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

In [23]:
model = "tf_semseg_model.h5"

In [29]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1, 2, 3))
    return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

def entr_and_dice(y_true, y_pred, **kwargs):
    return tf.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    # return K.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, [1, 2, 3]) + K.sum(y_pred, [1, 2, 3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return dice

In [32]:
reconstructed_model = keras.models.load_model(model, custom_objects={'entr_and_dice': entr_and_dice, 'iou_coef': iou_coef, 'dice_coef':dice_coef})